# Overview

Before we got our own fine-tuned LLMs. How we can make our basline LLM in a higher accuracy? One of the answers might be **Retrieval Augmented Generation(RAG)**. RAG enables LLMs reach beyond their static knowledge and grab real-time information quite easily. RAG taps into external sources, like internal company documents, to understand the context of our queries a bit better.

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/064/408/832/155/132/small/3f7a8bd57563298f.webp)

In [ ]:
%%capture
!pip install transformers==4.38.2
!pip install accelerate==0.27.2
!pip install datasets==2.18.0
!pip install peft==0.9.0
!pip install bitsandbytes==0.42.0

In [ ]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))


os.environ["MODEL_NAME"] = "google/gemma-2b-it"
os.environ["DATASET"] = ""

torch.backends.cudnn.deterministic=True
# https://github.com/huggingface/transformers/issues/28731
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"),
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

model.config.eos_token_id=tokenizer.eos_token_id
model.gradient_checkpointing_enable() # reducing memory usage


# Reference List

* https://www.promptingguide.ai/research/rag
* https://blog.gopenai.com/retrieval-augmented-generation-rag-585aa903d6bd
* https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-rag.ipynb